<h2>Importing Libraries</h2>

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

<h2>Load the Pre-trained Model and Tokenizer<h2>

In [7]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")
model.eval() # sets the model into evaluation mode, disabling certain layers (like dropout) used during training

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\Doctor Computers\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Doctor Computers\.cache\huggingface\hub\models--google-bert--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [9]:
sentence = "The capital of France is [MASK]."

In [11]:
inputs = tokenizer(sentence, return_tensors="pt") # Converts the tokenized output into PyTorch tensors (so it can be fed to the model).

<h2>Predict the Masked Token</h2>                                     
torch.no_grad(): Ensures that no gradients are calculated during the prediction (since we're not training the model).                               
model(**inputs): Feeds the tokenized input into the model to get predictions.                                                                       
outputs.logits: The model returns raw predictions (logits) for each token.                                                           
                These logits represent the model's confidence for each word in the vocabulary for every position in the input sequence.

In [17]:
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits

<h1>Find the Index of [MASK]</h1>
inputs.input_ids: The token IDs representing the input sentence.                                                                 
tokenizer.mask_token_id: The token ID for the special [MASK] token.                                                                              
.nonzero(as_tuple=True)[1]: Finds the position (index) of the [MASK] token in the input.

In [21]:
mask_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

<h1>Get the Most Likely Prediction</h1>                                                                                   
predictions[0, mask_index]: Extracts the logits for the [MASK] token position.                                                          
.argmax(dim=1): Finds the index of the highest probability for the [MASK] token (i.e., the most likely word).                                     

In [26]:
predicted_token_id = predictions[0, mask_index].argmax(dim=1)

<h1>Decode the Predicted Token</h1>                                                                                 
tokenizer.decode(predicted_token_id): Converts the predicted token ID back into a human-readable word.

In [29]:
predicted_word = tokenizer.decode(predicted_token_id)

In [31]:
print("Predicted word for [MASK]:", predicted_word)

Predicted word for [MASK]: paris
